In [0]:
# TODO: Start creating notebook, convert to a .py notebook (the ipynb are default and are stupid)
# TODO: We need an external volume with metadata files or we ingest the metadata from the pdfs, that might be easier.
# TODO: GRant access to the files we have for test to Brennan (I may have already, check)
# TODO: Ingest the metadata to bronze (use autoloader)
# TODO: Stream the appended records in bronze to silver, and on the way we're going to use forEachBatch to fetch a pdf, open it, count the pages, and optionally (maybe a different routine) trim to a certain number of pages. Keep it simple. Write the files to a volume OR to the silver delta table (we can think about the best way).
# TODO: For agentbricks, figure out if you can grab a binary column from a delta table or have to go to avolume.
# TODO: Paramaterize for catalog, schema
# TODO: hardcode table and volume names
# dbxmetagen.default.pfizer_files_del
# TODO: look at the options for reading pdfs and profiling them. What's out there, we want to count pages, and optionally trim. So ,what tools should we use for that, if any?

In [0]:
# import os
# import sys

In [0]:
dbutils.widgets.text("catalog", "dbxmetagen")
dbutils.widgets.text("source_schema", "default")
dbutils.widgets.text("dest_schema", "trimmed_pdfs")
dbutils.widgets.text("source_volume", "eln_pdfs")
dbutils.widgets.text("dest_volume", "trimmed_pdfs")

catalog = dbutils.widgets.get("catalog")
source_schema = dbutils.widgets.get("source_schema")
dest_schema = dbutils.widgets.get("dest_schema")
source_volume = dbutils.widgets.get("source_volume")
dest_volume = dbutils.widgets.get("dest_volume")

In [0]:
# import other things here

In [0]:
# Define functions and classes here
# TODO: function to use in forEachBatch that profiles PDFs. We could use multiple steps, but if we can cleanly put this in a single function it would be nice.

In [0]:
# Bronze

df = spark.readStream.format("cloudFiles").option("cloudFiles.format", "binary").load(f"/Volumes/{catalog}/{source_schema}/{source_volume}").select("metadata.*").trigger("availableNow")

df.writeStream.format("delta").option("checkpointLocation", f"/Volumes/{catalog}/{dest_schema}/_checkpoints/bronze_pdf_metadata").toTable(f"{catalog}.{dest_schema}.bronze_pdf_metadata")

In [0]:
# Silver

df.readStream.table(f"{catalog}.{dest_schema}.bronze_pdf_metadata").writeStream.foreachBatch(lambda df, epoch: print(df)).start()